In [1]:
from experiments.cubes_2d import get_dataloader, create_model


/mnt/antares_raid/home/jonasb/mambaforge/envs/wscrl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/antares_raid/home/jonasb/Projects/wscrl/repo/experiments/cubes_2d.py:50: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(


In [2]:
from ws_crl.posthoc_graph_learning.enco import run_enco
import logging
from omegaconf import OmegaConf
import torch
from pathlib import Path

logger = logging.getLogger()

# EXP_DIR = Path("/mnt/raid/ni/jonasb/wscrl/saved/sbd_complete")
EXP_DIR = Path(
    "/mnt/raid/ni/jonasb/wscrl/son_causal_variables_2dcubes_allow_collisions_10k_0.5_child_noise"
)


In [3]:
model_path = EXP_DIR / "models" / "model.pt"
cfg_path = EXP_DIR / "config.yml"

cfg = OmegaConf.load(cfg_path)

model = create_model(cfg)
model.load_state_dict(torch.load(model_path))
model.eval()


ILCM(
  (scm): MLPImplicitSCM(
    (solution_functions): ModuleList(
      (0-5): 6 x ConditionalAffineScalarTransform(
        (param_net): Sequential(
          (0): Linear(in_features=12, out_features=100, bias=True)
          (1): ReLU()
          (2): Linear(in_features=100, out_features=100, bias=True)
          (3): ReLU()
          (4): Linear(in_features=100, out_features=2, bias=True)
        )
      )
    )
    (base_density): StandardNormal()
  )
  (intervention_prior): InterventionPrior()
  (encoder): GaussianEncoder(
    (net): Sequential(
      (0): Linear(in_features=6, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): ReLU()
      (6): Linear(in_features=64, out_features=64, bias=True)
      (7): ReLU()
      (8): Linear(in_features=64, out_features=64, bias=True)
      (9): ReLU()
    )
    (mean_head): Linear(in_features

In [4]:
def eval_enco_graph(cfg, model, partition="train"):
    """Post-hoc graph evaluation with ENCO"""

    # Only want to do this for ILCMs
    if cfg.model.type not in ["intervention_noise_vae", "alt_intervention_noise_vae"]:
        return {}

    # Let's skip this for large latent spaces
    if cfg.model.dim_z > 8:
        return {}

    logger.info("Evaluating learned graph with ENCO")

    model.eval()
    device = torch.device(cfg.training.device)
    cpu = torch.device("cpu")
    model.to(device)

    # Load data and compute causal variables
    dataloader = get_dataloader(cfg, partition, cfg.eval.batchsize)
    z0s, z1s, interventions = [], [], []

    with torch.no_grad():
        for x0, x1, *_ in dataloader:
            x0, x1 = x0.to(device), x1.to(device)
            _, _, _, _, e0, e1, _, _, intervention = model.encode_decode_pair(
                x0.to(device), x1.to(device)
            )
            z0 = model.scm.noise_to_causal(e0)
            z1 = model.scm.noise_to_causal(e1)

            z0s.append(z0.to(cpu))
            z1s.append(z1.to(cpu))
            interventions.append(intervention.to(cpu))

        z0s = torch.cat(z0s, dim=0).detach()
        z1s = torch.cat(z1s, dim=0).detach()
        interventions = torch.cat(interventions, dim=0).detach()

    # Run ENCO
    adjacency_matrix = (
        run_enco(z0s, z1s, interventions, lambda_sparse=cfg.eval.enco_lambda, device=device)
        .cpu()
        .detach()
    )

    return adjacency_matrix

    # Package as dict
    # results = {
    #     f"enco_graph_{i}_{j}": adjacency_matrix[i, j].item()
    #     for i in range(model.dim_z)
    #     for j in range(model.dim_z)
    # }
    # return results


In [5]:
eval_enco_graph(cfg, model, partition=cfg.eval.eval_partition)


tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]])